# Compare fine mapped variants with GWAS catalog

In [2]:
setwd("/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc")

source("script/functions.R"  )


In [93]:
dir.create("pipeline/eqtls.1.6.compare_with_catalog", showWarnings = FALSE)


In [7]:
fm2cat   =              fread("pipeline/eqtls.0.1.coloc_gwas/fm2cat.txt"  , sep = "\t", header = TRUE, data.table = FALSE)
manifest = add_rownames(fread("pipeline/eqtls.0.1.coloc_gwas/manifest.txt", sep = "\t", header = TRUE, data.table = FALSE))
finemap  = readRDS           ("pipeline/eqtls.0.1.coloc_gwas/finemap_clean_list.rds")


In [73]:
assembly_rep           = fread("/frazer01/reference/public/dbsnp_155/GCF_000001405.13_GRCh37_assembly_report.txt", sep = "\t", header = FALSE, data.table = FALSE, skip = 35)[,c(10, 7)]
colnames(assembly_rep) = c("chrom", "refseq")
assembly_rep$chrom     = sub("chr", "", assembly_rep$chrom)

In [185]:
ldmat_list = readRDS("pipeline/eqtls.0.1.coloc_gwas/ldmat_catalog_list.rds")

In [188]:
catalog_start           = fread("/frazer01/reference/public/gwas_catalog/alternative.1", sep = "\t", header = TRUE, data.table = FALSE, quote = "")
#catalog_start           = catalog_start[, c("SNPS", "PUBMEDID", "STUDY", "MAPPED_TRAIT", "CHR_ID", "P-VALUE", "PVALUE_MLOG", "OR or BETA", "95% CI (TEXT)")]
#colnames(catalog_start) = c("rsid", "pmid", "study", "trait", "chrom", "pval", "logp", "beta", "confint")
#catalog_start$pval      = as.numeric(catalog_start$pval)
#catalog                 = catalog_start[ catalog_start$rsid %in% signals2fm$rsid, ]

str(catalog_start)

'data.frame':	257947 obs. of  38 variables:
 $ DATE ADDED TO CATALOG     : chr  "2019-03-28" "2019-03-28" "2019-03-28" "2019-03-28" ...
 $ PUBMEDID                  : int  30586737 30586737 30586737 30586737 30586737 30586737 30586737 30586737 30586737 30586737 ...
 $ FIRST AUTHOR              : chr  "Sabater-Lleal M" "Sabater-Lleal M" "Sabater-Lleal M" "Sabater-Lleal M" ...
 $ DATE                      : chr  "2018-11-20" "2018-11-20" "2018-11-20" "2018-11-20" ...
 $ JOURNAL                   : chr  "Circulation" "Circulation" "Circulation" "Circulation" ...
 $ LINK                      : chr  "www.ncbi.nlm.nih.gov/pubmed/30586737" "www.ncbi.nlm.nih.gov/pubmed/30586737" "www.ncbi.nlm.nih.gov/pubmed/30586737" "www.ncbi.nlm.nih.gov/pubmed/30586737" ...
 $ STUDY                     : chr  "Genome-Wide Association Trans-Ethnic Meta-Analyses Identifies Novel Associations Regulating Coagulation Factor "| __truncated__ "Genome-Wide Association Trans-Ethnic Meta-Analyses Identifies Novel Asso

In [187]:
str(ldmat_list[1:3])

List of 3
 $ chr1 :List of 2
  ..$ r2:'data.frame':	418 obs. of  418 variables:
  .. ..$ rs1886773  : num [1:418] 1 0.661 0.186 0.002 0 0.001 0.002 0.002 0.001 0.001 ...
  .. ..$ rs307359   : num [1:418] 0.661 1 0.312 0.005 0 0.002 0.006 0.006 0.006 0.004 ...
  .. ..$ rs1240716  : num [1:418] 0.186 0.312 1 0.042 0.001 0.014 0.042 0.049 0.058 0.058 ...
  .. ..$ rs3765908  : num [1:418] 0.002 0.005 0.042 1 0.181 0.003 0.016 0.018 0.017 0.017 ...
  .. ..$ rs374715536: num [1:418] 0 0 0.001 0.181 1 0 0.001 0.001 0.003 0.002 ...
  .. ..$ rs1014988  : num [1:418] 0.001 0.002 0.014 0.003 0 1 0.657 0.638 0.548 0.009 ...
  .. ..$ rs139385870: num [1:418] 0.002 0.006 0.042 0.016 0.001 0.657 1 0.958 0.751 0.02 ...
  .. ..$ rs2076328  : num [1:418] 0.002 0.006 0.049 0.018 0.001 0.638 0.958 1 0.778 0.023 ...
  .. ..$ rs7546498  : num [1:418] 0.001 0.006 0.058 0.017 0.003 0.548 0.751 0.778 1 0.029 ...
  .. ..$ rs262695   : num [1:418] 0.001 0.004 0.058 0.017 0.002 0.009 0.02 0.023 0.029 1 ...
  .. .

In [ ]:
get_gwas_data = function(coord, gwas_file)
{
    my_head              = colnames(fread(cmd = paste("zcat", gwas_file, "|", "head -n 2"), sep = "\t", header = TRUE, data.table = FALSE))
    gwas_data            = suppressWarnings(tabix.read.table(gwas_file, coord, col.names = TRUE, stringsAsFactors = FALSE))
    colnames(gwas_data)  = my_head
    gwas_data$variant_id = paste("VAR", gwas_data$chr, gwas_data$pos, gwas_data$ref, gwas_data$alt, sep = "_")
    rownames(gwas_data)  = gwas_data$variant_id
    
    return(gwas_data)
}

read_dbsnp = function(coord)
{
    chrom            = unlist(strsplit(coord, ":"))[[1]]
    refseq           = assembly_rep[ assembly_rep$chrom == chrom, "refseq"]
    coord2           = paste(refseq, unlist(strsplit(coord, ":"))[[2]], sep = ":")
    indata           = suppressWarnings(tabix.read.table("/frazer01/reference/public/dbsnp_155/GCF_000001405.25.gz", coord2, col.names = TRUE, stringsAsFactors = FALSE))
    colnames(indata) = tolower(colnames(indata))
    indata$af        = unlist(lapply(indata$info, function(info)
    {
        x = unlist(strsplit(info, ";"))
        x = x[grepl("FREQ", x) == TRUE]
        
        if(length(x) == 0){x = ""}
        if(length(x) >  1){x = x[[1]]}

        return(x)
    }))

    indata        = indata[,c("pos", "id", "ref", "alt", "af")]
    indata$var_id = paste("VAR", chrom, indata$pos, indata$ref, unlist(lapply(indata$alt, function(x){unlist(strsplit(x, ","))[[1]]})), sep = "_")

    return(indata)
}

add_beta = function(ii, fm2cat, manifest, finemap)
{
    message(ii, appendLF = FALSE)
    
    transcript_id = fm2cat[ii, "transcript_id"]
    type          = fm2cat[ii, "type"         ]
    trait         = fm2cat[ii, "gwas"         ]
    id            = fm2cat[ii, "id"           ]
    rsid          = fm2cat[ii, "rsid"         ]
    overlap       = fm2cat[ii, "overlap"      ]
    cat           = fm2cat[ii, "cat"          ]
    
    fm          = finemap[[paste(transcript_id, type, trait, sep = ":")]]
    qtl         = fm[["qtl"  ]]
    coord       = fm[["coord"]]
    var_catalog = ""
    gwas        = get_gwas_data(coord, manifest[trait, "sumstat_file"])
    
    if(nrow(qtl[ qtl$rsid == overlap, ]) >  0){var_catalog = qtl[ qtl$rsid == overlap, "id"]}
    
    dbsnp = read_dbsnp(coord)
    dbsnp = list(qtl = dbsnp[ dbsnp$id == rsid,], catalog = dbsnp[ dbsnp$id == overlap,])
    out1  = gwas[ gwas$variant_id == id,]
    out2  = gwas[ gwas$variant_id == var_catalog,]
                 
    #if(var_catalog != ""){out2 = gwas[ gwas$variant_id == var_catalog,]}
    #if(var_catalog == ""){out2 = NULL}
    
    out = list(fm2cat = fm2cat[ii,], dbsnp = dbsnp, gwas = list(qtl = out1, catalog = out2))
    
    return(out)
}

ii = 1
by_locus        = lapply(1:nrow(fm2cat), function(ii){add_beta(ii, fm2cat, manifest, finemap)})
names(by_locus) = paste(fm2cat$transcript_id, fm2cat$type, fm2cat$gwas, sep = ":")

saveRDS(by_locus, "pipeline/eqtls.1.6.compare_with_catalog/by_locus.rds")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167


In [163]:
get_gwas_by_pop = function(pop, type, x)
{
    indata = x[["gwas"]][[type]]
    af_col = "af"

    if(!paste(af_col, pop, sep = "_") %in% colnames(indata)){af_col = "af_controls"}
    
    out           = indata[,c("pos", "ref", "alt", "variant_id", paste(c(af_col, "beta", "se", "pval"), pop, sep = "_"))]
    colnames(out) = c("pos", "ref", "alt", "variant_id", "af", "beta", "se", "pval")
    
    if(nrow(out) == 0){out[1,] = NA}
    
    out$source    = type
    out$pop       = pop
    
    return(out)
}

reorganize_locus = function(name, by_locus, manifest)
{
    #message(name, appendLF = FALSE)
    
    in_names      = unlist(strsplit(name, ":"))
    transcript_id = in_names[[1]]
    type          = in_names[[2]]
    trait         = in_names[[3]]
    x             = by_locus[[name]]
    pops          = c("meta", unlist(strsplit(manifest[trait, "pops"], ",")))
   
    gwas_qtl  = as.data.frame(rbindlist(lapply(pops, function(pop){get_gwas_by_pop(pop, "qtl"    , x)})), stringsAsFactors = FALSE)
    gwas_gwas = as.data.frame(rbindlist(lapply(pops, function(pop){get_gwas_by_pop(pop, "catalog", x)})), stringsAsFactors = FALSE)
    gwas      = rbind(gwas_qtl, gwas_gwas)
    out       = suppressWarnings(cbind(fm2cat[name, c("name", "transcript_id", "gene_id", "gene_name", "type", "gwas", "PP.H4.abf", "pp_snp", "cs", "fm", "r2", "d", "cat")], gwas))
    
    return(out)
}

fm2cat$name      = paste(fm2cat$transcript_id, fm2cat$type, fm2cat$gwas, sep = ":")
rownames(fm2cat) = fm2cat$name
var2pop          = as.data.frame(rbindlist(lapply(fm2cat$name, function(name){reorganize_locus(name, by_locus, manifest)})), stringsAsFactors = FALSE)

#name = "ENSG00000011523.14_5:0:icd10-I48-both_sexes"
#reorganize_locus(name, by_locus, manifest)

In [178]:
to_merge = c("name", "transcript_id", "gene_id", "gene_name", "type", "gwas", "PP.H4.abf", "pp_snp", "cs", "fm", "r2", "d", "cat", "pop")
to_cols  = c("af", "beta", "se", "pval")
var2test = merge(var2pop[ var2pop$source == "qtl"    , c(to_merge, to_cols)],
                 var2pop[ var2pop$source == "catalog", c(to_merge, to_cols)],
                 by       = to_merge,
                 suffixes = c("_qtl", "_gwas"),
                 all      = TRUE
                )
var2test = var2test[ var2test$pop == "meta",]

In [180]:
head(var2test[ var2test$cat == "high D'",])

,name,transcript_id,gene_id,gene_name,type,gwas,PP.H4.abf,pp_snp,cs,fm,⋯,cat,pop,af_qtl,beta_qtl,se_qtl,pval_qtl,af_gwas,beta_gwas,se_gwas,pval_gwas
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<int>,<lgl>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
25,ENSG00000011523.14_5:0:icd10-I48-both_sexes,ENSG00000011523.14_5,ENSG00000011523.14_5,CEP68,0,icd10-I48-both_sexes,0.9984514,0.9999999,1,TRUE,⋯,high D',meta,0.38210,-0.07710,0.011360,1.164e-11,0.3759,-0.07420,0.011330,5.857e-11
31,ENSG00000031003.10_5:0:continuous-102-both_sexes-irnt,ENSG00000031003.10_5,ENSG00000031003.10_5,FAM13B,0,continuous-102-both_sexes-irnt,0.9629795,0.3949898,17,TRUE,⋯,high D',meta,0.17440,-0.03906,0.003258,4.006e-33,0.1860,-0.03613,0.003174,5.224e-30
36,ENSG00000031003.10_5:0:icd10-I48-both_sexes,ENSG00000031003.10_5,ENSG00000031003.10_5,FAM13B,0,icd10-I48-both_sexes,0.9628775,0.2415948,18,TRUE,⋯,high D',meta,0.17500,0.11610,0.014410,7.630e-16,0.3199,0.09471,0.011790,9.622e-16
46,ENSG00000064989.13_5:0:continuous-102-both_sexes-irnt,ENSG00000064989.13_5,ENSG00000064989.13_5,CALCRL,0,continuous-102-both_sexes-irnt,0.9840027,0.7047911,6,TRUE,⋯,high D',meta,0.19540,0.03923,0.003121,3.180e-36,NA,NA,NA,NA
60,ENSG00000068878.15_5:1:continuous-PP-both_sexes-combined_medadj_irnt,ENSG00000068878.15_5,ENSG00000068878.15_5,PSME4,1,continuous-PP-both_sexes-combined_medadj_irnt,0.9262523,0.2436688,25,TRUE,⋯,high D',meta,0.07521,0.02462,0.004160,3.272e-09,NA,NA,NA,NA
81,ENSG00000075420.13_4:0:continuous-102-both_sexes-irnt,ENSG00000075420.13_4,ENSG00000075420.13_4,FNDC3B,0,continuous-102-both_sexes-irnt,0.9731374,0.4050497,4,TRUE,⋯,high D',meta,0.40110,0.02347,0.002541,2.628e-20,NA,NA,NA,NA


In [182]:
name = "ENSG00000064989.13_5:0:continuous-102-both_sexes-irnt"
str(fm2cat[name,])

'data.frame':	1 obs. of  26 variables:
 $ gene_id      : chr "ENSG00000064989.13_5"
 $ gene_name    : chr "CALCRL"
 $ chrom        : chr "chr2"
 $ id           : chr "VAR_2_188165166_A_G"
 $ transcript_id: chr "ENSG00000064989.13_5"
 $ nsnps        : int 1538
 $ PP.H0.abf    : num 2.89e-32
 $ PP.H1.abf    : num 0.00489
 $ PP.H2.abf    : num 7.16e-32
 $ PP.H3.abf    : num 0.0111
 $ PP.H4.abf    : num 0.984
 $ gwas         : chr "continuous-102-both_sexes-irnt"
 $ pop          : chr "meta"
 $ phenotype    : chr "rna"
 $ type         : int 0
 $ pp_snp       : num 0.705
 $ rsid         : chr "rs17705966"
 $ cluster      : chr "2_14"
 $ cs           : int 6
 $ fm           : logi TRUE
 $ overlap      : chr "rs111641830"
 $ r2           : num 0.001
 $ d            : num 1
 $ pmid         : chr "31113495"
 $ cat          : chr "high D'"
 $ name         : chr "ENSG00000064989.13_5:0:continuous-102-both_sexes-irnt"


In [184]:
str(ldmat_list)

List of 21
 $ chr1 :List of 2
  ..$ r2:'data.frame':	14 obs. of  14 variables:
  .. ..$ rs10864434: num [1:14] 1 0.442 0.001 0.008 0.002 0 0 0.007 0.007 0.009 ...
  .. ..$ rs709209  : num [1:14] 0.442 1 0.002 0.01 0.002 0 0 0.008 0.01 0.008 ...
  .. ..$ rs11247639: num [1:14] 0.001 0.002 1 0.013 0.001 0.005 0.001 0.007 0.002 0.001 ...
  .. ..$ rs67631072: num [1:14] 0.008 0.01 0.013 1 0.002 0 0.001 0.007 0.011 0.002 ...
  .. ..$ rs17343290: num [1:14] 0.002 0.002 0.001 0.002 1 0 0 0.004 0.004 0.002 ...
  .. ..$ rs1416706 : num [1:14] 0 0 0.005 0 0 1 0 0.001 0 0.001 ...
  .. ..$ rs11207420: num [1:14] 0 0 0.001 0.001 0 0 1 0 0.001 0 ...
  .. ..$ rs9428222 : num [1:14] 0.007 0.008 0.007 0.007 0.004 0.001 0 1 0.012 0.002 ...
  .. ..$ rs4970923 : num [1:14] 0.007 0.01 0.002 0.011 0.004 0 0.001 0.012 1 0.027 ...
  .. ..$ rs2305815 : num [1:14] 0.009 0.008 0.001 0.002 0.002 0.001 0 0.002 0.027 1 ...
  .. ..$ rs12138150: num [1:14] 0 0 0.014 0.004 0.003 0 0 0.005 0.002 0 ...
  .. ..$ rs213318

In [24]:
manifest

,trait_type,phenocode,pheno_sex,coding,modifier,description,description_more,coding_description,category,n_cases_full_cohort_both_sexes,⋯,lambda_gc_EAS,lambda_gc_EUR,lambda_gc_MID,filename,sumstat_file,id,name,n_cases_meta,n_controls_meta,color
,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<lgl>,<chr>,<int>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>
continuous-102-both_sexes-irnt,continuous,102,both_sexes,NA,irnt,"Pulse rate, automated reading",This is the pulse rate measured during the automated blood pressure readings.,NA,UK Biobank Assessment Centre > Physical measures > Blood pressure,457245,⋯,1.0230,1.4605,1.0136,continuous-102-both_sexes-irnt.tsv.bgz,/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/1.1.sumstats/sumstats/continuous-102-both_sexes-irnt.tsv.bgz,continuous-102-both_sexes-irnt,Pulse rate,416959,0,#201158
continuous-12340-both_sexes-irnt,continuous,12340,both_sexes,NA,irnt,QRS duration,QRS duration during ECG measurement,NA,"UK Biobank Assessment Centre > Physical measures > ECG at rest, 12-lead",36352,⋯,1.0208,1.0516,1.0179,continuous-12340-both_sexes-irnt.tsv.bgz,/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/1.1.sumstats/sumstats/continuous-12340-both_sexes-irnt.tsv.bgz,continuous-12340-both_sexes-irnt,QRS duration,33242,0,#005E5E
continuous-PP-both_sexes-combined_medadj_irnt,continuous,PP,both_sexes,NA,combined_medadj_irnt,"Pulse pressure, combined automated + manual reading, adjusted by medication","SBP (4080, 93) - DBP (4079, 94), adjusted by the use of blood pressure lowering medication (6153_2 and 6177_2)",NA,,479101,⋯,0.9827,1.6217,1.0426,continuous-PP-both_sexes-combined_medadj_irnt.tsv.bgz,/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/1.1.sumstats/sumstats/continuous-PP-both_sexes-combined_medadj_irnt.tsv.bgz,continuous-PP-both_sexes-combined_medadj_irnt,Pulse pressure,436843,0,#578B21
icd10-I48-both_sexes,icd10,I48,both_sexes,NA,,I48 Atrial fibrillation and flutter,truncated: true,NA,Chapter IX Diseases of the circulatory system | I30-I52 Other forms of heart disease | I48 Atrial fibrillation and flutter,21286,⋯,NA,1.0907,NA,icd10-I48-both_sexes.tsv.bgz,/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/1.1.sumstats/sumstats/icd10-I48-both_sexes.tsv.bgz,icd10-I48-both_sexes,Atrial fibrillation,19554,416489,#E89E6B
phecode-411.2-both_sexes,phecode,411.2,both_sexes,NA,,Myocardial infarction,,NA,circulatory system,17259,⋯,NA,1.1037,1.0774,phecode-411.2-both_sexes.tsv.bgz,/frazer01/projects/CARDIPS/analysis/cardiac_gwas_coloc/pipeline/1.1.sumstats/sumstats/phecode-411.2-both_sexes.tsv.bgz,phecode-411.2-both_sexes,Myocardial infarction,15816,397202,#FFCEF4
